In [1]:
!git clone https://github.com/matterport/Mask_RCNN.git
!pip3 install -r requirements.txt
!pip install Cython
!git clone https://github.com/waleedka/coco.git

!pip install -U setuptools
!pip install -U wheel
!make install -C coco/PythonAPI


Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.84 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (570/570), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Cloning into 'coco'...
remote: Enumerating objects: 904, done.
remote: Total 904 (delta 0), reused 0 (delta 0), pack-reused 904
Receiving objects: 100% (904/904), 10.39 MiB | 7.08 MiB/s, done.
Resolving deltas: 100% (539/539), done.
Requirement already up-to-date: setuptools in /home/erbachj/MaskRCNN/lib/python3.6/site-packages (46.0.0)
Requirement already up-to-date: wheel in /home/erbachj/MaskRCNN/lib/python3.6/site-packages (0.34.2)
make: Entering directory '/home/erbachj/MaskRCNN/Mask_RCNN/Notebook/coco/PythonAPI'
# install pycocotools to the Python site-packages
python setup.py build_ext install
Compiling pycocotools/_mask.pyx because it changed.
[1/1

rm -rf build
make: Leaving directory '/home/erbachj/MaskRCNN/Mask_RCNN/Notebook/coco/PythonAPI'


In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
#from pycocotools.coco import COCO


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco
#from samples.coco import coco
print('finished')

/home/erbachj/MaskRCNN/Mask_RCNN


Using TensorFlow backend.


**Import Mask R-CNN**

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
print(MODEL_DIR)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
print(COCO_MODEL_PATH)
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [ ]:
batch_size = 2
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    NAME = 'Coco'
    #BATCH_SIZE = batch_size

config = InferenceConfig()
config.display()

In [ ]:

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# Load COCO dataset
COCO_DIR = os.path.join(ROOT_DIR,'coco')
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

IMAGE_DIR = os.path.join(ROOT_DIR, "Test_Data")
IMAGE_SAVE_DIR = os.path.join(ROOT_DIR, "Image_Results")
img = cv2.imread(os.path.join(IMAGE_DIR, "Test.jpg"))
image = []
image.append(img)
print(len(image))
print(config.BATCH_SIZE)
r = model.detect(image, verbose=0)
visualize.save_image(
                image_name = name, image = image, boxes = r['rois'], masks = r['masks'], class_ids = r['class_ids'], \
                class_names = class_names, scores = r['scores'], save_dir = IMAGE_SAVE_DIR
            )

In [ ]:
VIDEO_DIR = os.path.join(ROOT_DIR, "Test_Data")
VIDEO_SAVE_DIR = os.path.join(ROOT_DIR, "Results")
print(VIDEO_DIR)
capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, 'Test-Clip2.mp4'))  

frame_count = 0
frames = []

while True:
    ret, frame = capture.read()
    # Bail out when the video file ends
    if not ret:
        break        
    # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    if len(frames) == batch_size:
        results = model.detect(frames, verbose=0)
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            name = '{0}.jpg'.format(frame_count + i - batch_size)
            visualize.save_image(
                image_name = name, image = frame, boxes = r['rois'], masks = r['masks'], class_ids = r['class_ids'], \
                class_names = class_names, scores = r['scores'], save_dir = VIDEO_SAVE_DIR
            )
        # Clear the frames array to start the next batch
        frames = []